In [11]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("data.json").read()
intents = json.loads(data_file)

In [12]:
# preprocessing the json data
# tokenization
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     c:\Users\USER\anaconda3\envs\chatbot\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     c:\Users\USER\anaconda3\envs\chatbot\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('labels.pkl','wb'))

139 documents
30 classes ['adaptasi_iklim', 'buka_website', 'edukasi_pesisir', 'edukasi_program', 'erosi_pengertian', 'erosi_penyebab', 'erosi_solusi', 'estuari_manfaat', 'estuari_pengertian', 'fallback', 'goodbye', 'greeting', 'karang_kerusakan', 'karang_pelestarian', 'karang_pengertian', 'mangrove_kerusakan', 'mangrove_manfaat', 'mangrove_pengertian', 'perikanan_masalah', 'perikanan_pengertian', 'perikanan_upaya', 'perubahan_iklim_pengertian', 'pesisir_manfaat', 'pesisir_masalah', 'pesisir_pengertian', 'plastik_pengertian', 'plastik_solusi', 'polusi_laut_pengertian', 'polusi_laut_sumber', 'thanks']
147 unique lemmatized words ['abrasi', 'ada', 'adaptasi', 'aku', 'ancaman', 'apa', 'assalamualaikum', 'bagaimana', 'bagi', 'banyak', 'berasal', 'berbahaya', 'berkelanjutan', 'bisa', 'bleaching', 'bot', 'buka', 'bye', 'cabut', 'cara', 'change', 'chatbot', 'climate', 'contoh', 'coral', 'dada', 'dadah', 'dari', 'definisi', 'di', 'diedukasi', 'digunakan', 'dimaksud', 'disebabkan', 'dulu', 'edu

In [14]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


In [15]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('models.h5') # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0504 - loss: 3.4044       
Epoch 2/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1799 - loss: 3.3163 
Epoch 3/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1511 - loss: 3.2078 
Epoch 4/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1511 - loss: 3.1862 
Epoch 5/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1583 - loss: 3.0947 
Epoch 6/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1799 - loss: 3.0248     
Epoch 7/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1871 - loss: 2.8771     
Epoch 8/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2230 - loss: 2.7645 
Epoch 9/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2590 - loss: 2.5746     
Epoch 10/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3237 - loss: 2.4436 
Epoch 11/200
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3741 - loss: 2.2589 
Epoch 12/200
28/28 ━━━━━━━━━━━━━━━



**************************************************

Model Created Successfully!
